# Import

In [92]:
import sys

import pandas as pd

sys.path.append("../")
import datetime
from time import sleep

import requests
from logzero import logger
from pandas.testing import assert_frame_equal

from src.config import *
from src.local_config import *

In [85]:
req_params

{'applicationId': '1076096003993096283',
 'affiliateId': '21ca5228.e21e9760.21ca5229.ab3a477c',
 'format': 'json',
 'formatVersion': '2',
 'keyword': 'margiela bag 中古',
 'hits': 30,
 'sort': '-itemPrice',
 'page': 1,
 'minPrice': 100,
 'hogehoge': 'hogehoge'}

# fetch_products

In [83]:
def fetch_products(brand, item):
    keyword = f"{brand} {item} 中古"
    df = pd.DataFrame(columns=WANT_ITEMS)

    for page in range(1, MAX_PAGE + 1):
        req_params.update({"page": page, "keyword": keyword})
        response = requests.get(REQ_URL, req_params)
        if response.status_code != 200:
            logger.error(f"ErrorCode -> {response.status_code}\nPage -> {page}")
            continue

        data = response.json()
        if data["hits"] == 0:
            logger.info("No more products found.")
            break

        tmp_df = pd.DataFrame(data["Items"])[WANT_ITEMS]
        df = pd.concat([df, tmp_df], ignore_index=True)
        logger.info(f"Page {page} processed.")
        sleep(1)  # Avoid hitting API rate limit

    return df

## テストケース1

In [79]:
# 以下のケースでitemName、itemPrice、itemUrlが30行返却されるか。先頭、後方5行の外観が想定通りか。
# brandが"margiela"
# itemが"bag"
# MAX_PAGEが1

In [84]:
brand = "margiela"
item = "bag"
MAX_PAGE = 1
df = fetch_products(brand, item)
display(df.head())
display(df.tail())
assert df.columns.tolist() == ["itemName", "itemPrice", "itemUrl"]
assert df.shape[0] == 30

[I 240501 23:53:55 2207717512:19] Page 1 processed.


,itemName,itemPrice,itemUrl
0,【最大3万円クーポン 5/1~3】【天白】【GLOBE TROTTER】グローブトロッター ...,396550,https://hb.afl.rakuten.co.jp/hgc/g00ryq09.disk...
1,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ク...,231000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
2,【美品】メゾン マルジェラ 5AC ミニバッグ ショルダーバッグ,217800,https://hb.afl.rakuten.co.jp/hgc/g00rzsu9.disk...
3,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ホ...,214500,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
4,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ク...,209000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...


,itemName,itemPrice,itemUrl
25,【新品同様】メゾン マルジェラ バッグ グラムスラム ホーボースモール,189000,https://hb.afl.rakuten.co.jp/hgc/g00rzsu9.disk...
26,メゾン マルジェラ MAISON MARGIELA 5AC ショルダーバッグ レザー/キャン...,187000,https://hb.afl.rakuten.co.jp/hgc/g00qp0n9.disk...
27,メゾン マルジェラ （Maison Margiela） 5AC マイクロ 2way ハンド ...,187000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
28,メゾン マルジェラ MAISON MARGIELA 5AC ミニバッグ ショルダーバッグ レ...,187000,https://hb.afl.rakuten.co.jp/hgc/g00qp0n9.disk...
29,メゾン マルジェラ 5AC マイクロ バッグ 【新品同様】,185000,https://hb.afl.rakuten.co.jp/hgc/g00rzsu9.disk...


## テストケース2

In [86]:
# 以下のケースでitemName、itemPrice、itemUrlがMAX_PAGES * hits - 1以上の行が返却されるか。先頭、後方5行の外観が想定通りか。
# brandが"margiela"
# itemが"bag"
# MAX_PAGEが30

In [88]:
brand = "margiela"
item = "bag"
MAX_PAGE = 30
df = fetch_products(brand, item)
display(df.head())
display(df.tail())
assert df.columns.tolist() == ["itemName", "itemPrice", "itemUrl"]
assert df.shape[0] > MAX_PAGE * (HITS_PER_PAGE - 1)

[I 240501 23:56:35 2207717512:19] Page 1 processed.
[I 240501 23:56:36 2207717512:19] Page 2 processed.
[I 240501 23:56:38 2207717512:19] Page 3 processed.
[I 240501 23:56:39 2207717512:19] Page 4 processed.
[I 240501 23:56:40 2207717512:19] Page 5 processed.
[I 240501 23:56:41 2207717512:19] Page 6 processed.
[I 240501 23:56:42 2207717512:19] Page 7 processed.
[I 240501 23:56:43 2207717512:19] Page 8 processed.
[I 240501 23:56:45 2207717512:19] Page 9 processed.
[I 240501 23:56:46 2207717512:19] Page 10 processed.
[I 240501 23:56:47 2207717512:19] Page 11 processed.
[I 240501 23:56:48 2207717512:19] Page 12 processed.
[I 240501 23:56:49 2207717512:19] Page 13 processed.
[I 240501 23:56:50 2207717512:19] Page 14 processed.
[I 240501 23:56:52 2207717512:19] Page 15 processed.
[I 240501 23:56:53 2207717512:19] Page 16 processed.
[I 240501 23:56:54 2207717512:19] Page 17 processed.
[I 240501 23:56:55 2207717512:19] Page 18 processed.
[I 240501 23:56:56 2207717512:19] Page 19 processed.
[I

,itemName,itemPrice,itemUrl
0,【最大3万円クーポン 5/1~3】【天白】【GLOBE TROTTER】グローブトロッター ...,396550,https://hb.afl.rakuten.co.jp/hgc/g00ryq09.disk...
1,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ク...,231000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
2,【美品】メゾン マルジェラ 5AC ミニバッグ ショルダーバッグ,217800,https://hb.afl.rakuten.co.jp/hgc/g00rzsu9.disk...
3,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ホ...,214500,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
4,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ク...,209000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...


,itemName,itemPrice,itemUrl
757,【中古】Maison Margiela◆トートバッグ/コットン/NVY/S55WC0074【...,8690,https://hb.afl.rakuten.co.jp/hgc/g00r3ce9.disk...
758,【中古】MM6 Maison MargielaPVC Print Tote Bag トートバ...,8250,https://hb.afl.rakuten.co.jp/hgc/g00q49j9.disk...
759,MM6 Maison Margiela(エムエムシックスメゾンマルジェラ) サイズ:36 1...,8250,https://hb.afl.rakuten.co.jp/hgc/g00s6ri9.disk...
760,【中古】MM6 Maison Margiela「Net Fabric Bag」 メッシュトー...,7150,https://hb.afl.rakuten.co.jp/hgc/g00q49j9.disk...
761,【中古】Maison Margiela◆トートバッグ/コットン/WHT/無地/S55WC00...,6490,https://hb.afl.rakuten.co.jp/hgc/g00r3ce9.disk...


AssertionError: 

In [89]:
df

,itemName,itemPrice,itemUrl
0,【最大3万円クーポン 5/1~3】【天白】【GLOBE TROTTER】グローブトロッター ...,396550,https://hb.afl.rakuten.co.jp/hgc/g00ryq09.disk...
1,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ク...,231000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
2,【美品】メゾン マルジェラ 5AC ミニバッグ ショルダーバッグ,217800,https://hb.afl.rakuten.co.jp/hgc/g00rzsu9.disk...
3,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ホ...,214500,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
4,メゾン マルジェラ （Maison Margiela） Glam Slam グラムスラム ク...,209000,https://hb.afl.rakuten.co.jp/hgc/g00rtpq9.disk...
...,...,...,...
757,【中古】Maison Margiela◆トートバッグ/コットン/NVY/S55WC0074【...,8690,https://hb.afl.rakuten.co.jp/hgc/g00r3ce9.disk...
758,【中古】MM6 Maison MargielaPVC Print Tote Bag トートバ...,8250,https://hb.afl.rakuten.co.jp/hgc/g00q49j9.disk...
759,MM6 Maison Margiela(エムエムシックスメゾンマルジェラ) サイズ:36 1...,8250,https://hb.afl.rakuten.co.jp/hgc/g00s6ri9.disk...
760,【中古】MM6 Maison Margiela「Net Fabric Bag」 メッシュトー...,7150,https://hb.afl.rakuten.co.jp/hgc/g00q49j9.disk...


# save_tweet_texts

In [90]:
def save_tweet_texts(brand, df):
    today = datetime.datetime.today().strftime("%Y%m%d")
    output_dir = Path(path_output_dir) / brand
    output_dir.mkdir(parents=True, exist_ok=True)

    for i, row in df.iterrows():
        tweet_text = f"アイテム名: {row['itemName']}\n価格: {row['itemPrice']}\nURL: {row['itemUrl']} PR"
        tweet_file_path = output_dir / f"tweet_{today}_{i}.txt"
        with open(tweet_file_path, "w") as file:
            file.write(tweet_text)
        logger.info(f"Saved: {tweet_file_path}")

In [93]:
path_output_dir = "../data/output/"
save_tweet_texts(brand, df)

[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_0.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_1.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_2.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_3.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_4.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_5.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_6.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_7.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_8.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_9.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/output/margiela/tweet_20240502_10.txt
[I 240502 00:05:30 1776146018:11] Saved: ../data/outp